In [269]:
import pandas as pd

taxi_zone = pd.read_csv('../data/taxi-zone-lookup.csv')
print(taxi_zone.head())
print(taxi_zone.shape)

uber_raw = pd.read_csv('../data/uber-raw-data-apr14.csv')
print(uber_raw.head())
print(uber_raw.shape)
print(uber_raw.dtypes)

   LocationID        Borough                     Zone
0           1            EWR           Newark Airport
1           2         Queens              Jamaica Bay
2           3          Bronx  Allerton/Pelham Gardens
3           4      Manhattan            Alphabet City
4           5  Staten Island            Arden Heights
(265, 3)
          Date/Time      Lat      Lon    Base
0  4/1/2014 0:11:00  40.7690 -73.9549  B02512
1  4/1/2014 0:17:00  40.7267 -74.0345  B02512
2  4/1/2014 0:21:00  40.7316 -73.9873  B02512
3  4/1/2014 0:28:00  40.7588 -73.9776  B02512
4  4/1/2014 0:33:00  40.7594 -73.9722  B02512
(564516, 4)
Date/Time     object
Lat          float64
Lon          float64
Base          object
dtype: object


In [270]:
print(uber_raw['Base'].value_counts())
print(uber_raw.isna().sum())

Base
B02682    227808
B02598    183263
B02617    108001
B02512     35536
B02764      9908
Name: count, dtype: int64
Date/Time    0
Lat          0
Lon          0
Base         0
dtype: int64


In [271]:
split_datetime = uber_raw.copy()
split_datetime['Date/Time'] = pd.to_datetime(split_datetime["Date/Time"])
split_datetime['date'] = split_datetime['Date/Time'].dt.date
split_datetime['time'] = split_datetime['Date/Time'].dt.time
split_datetime["annee"] = split_datetime['Date/Time'].dt.year
split_datetime["mois"] = split_datetime['Date/Time'].dt.month
split_datetime["jour"] = split_datetime['Date/Time'].dt.day
split_datetime["jour_semaine"] = split_datetime['Date/Time'].dt.dayofweek
split_datetime["heure"] = split_datetime['Date/Time'].dt.hour
split_datetime["minutes"] = split_datetime['Date/Time'].dt.minute
split_datetime["secondes"] = split_datetime['Date/Time'].dt.second

print(split_datetime.head())

            Date/Time      Lat      Lon    Base        date      time  annee  \
0 2014-04-01 00:11:00  40.7690 -73.9549  B02512  2014-04-01  00:11:00   2014   
1 2014-04-01 00:17:00  40.7267 -74.0345  B02512  2014-04-01  00:17:00   2014   
2 2014-04-01 00:21:00  40.7316 -73.9873  B02512  2014-04-01  00:21:00   2014   
3 2014-04-01 00:28:00  40.7588 -73.9776  B02512  2014-04-01  00:28:00   2014   
4 2014-04-01 00:33:00  40.7594 -73.9722  B02512  2014-04-01  00:33:00   2014   

   mois  jour  jour_semaine  heure  minutes  secondes  
0     4     1             1      0       11         0  
1     4     1             1      0       17         0  
2     4     1             1      0       21         0  
3     4     1             1      0       28         0  
4     4     1             1      0       33         0  


In [272]:
import plotly.express as px

fig = px.scatter_map(split_datetime[:20000],
               lat='Lat',
               lon='Lon',
               map_style="carto-positron",width=800)
fig.show()

In [273]:
# sauvegarde du DataFrame avec split des dates
split_datetime.to_csv('../data/echantillon.csv')

In [275]:

from sklearn.cluster import DBSCAN

#selection d'un échantillon de trois jour entre 8 et 10h
split_datetime = split_datetime[(split_datetime['jour'].between(1,2)) & (split_datetime['heure'].between(8,9))]
print(split_datetime.shape)
coords = split_datetime[["Lat", "Lon"]].to_numpy()
# DBSCAN
db = DBSCAN(eps=0.001, min_samples=5, metric="euclidean")  
labels = db.fit_predict(coords)

split_datetime["cluster"] = labels


print(split_datetime["cluster"].value_counts())

fig = px.scatter_map(split_datetime,
               lat='Lat',
               lon='Lon',
               map_style="carto-positron",
               width=800,
               color='cluster'
               )
fig.show()

(2895, 13)
cluster
-1      1416
 36       51
 16       42
 4        34
 49       34
        ... 
 120       5
 128       5
 125       5
 132       5
 127       5
Name: count, Length: 135, dtype: int64
